In [ ]:
Table: Logs

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| num         | varchar |
+-------------+---------+
id is the primary key for this table.
 

Write an SQL query to find all numbers that appear at least three times consecutively.

Return the result table in any order.

The query result format is in the following example:

 

Logs table:
+----+-----+
| Id | Num |
+----+-----+
| 1  | 1   |
| 2  | 1   |
| 3  | 1   |
| 4  | 2   |
| 5  | 1   |
| 6  | 2   |
| 7  | 2   |
+----+-----+

Result table:
+-----------------+
| ConsecutiveNums |
+-----------------+
| 1               |
+-----------------+
1 is the only number that appears consecutively for at least three times.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.appName("consecutive_numbers").master("local[*]").getOrCreate()

schema = ["id", "num"]
data = [(1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 2), (7, 2)]

logs_df = spark.createDataFrame(data, schema)

logs_df.show()

+---+---+
| id|num|
+---+---+
|  1|  1|
|  2|  1|
|  3|  1|
|  4|  2|
|  5|  1|
|  6|  2|
|  7|  2|
+---+---+



In [5]:
from pyspark.sql.functions import lag, lead, col
from pyspark.sql.window import Window

windowSpec = Window.partitionBy().orderBy("id")

lag_lead_df = logs_df.withColumn("lag", lag("num").over(windowSpec)).withColumn("lead", lead("num").over(windowSpec))

lag_lead_df.show()

lag_lead_df.where("num = lag = lead").selectExpr("num as ConsecutiveNums").show()

+---+---+----+----+
| id|num| lag|lead|
+---+---+----+----+
|  1|  1|null|   1|
|  2|  1|   1|   1|
|  3|  1|   1|   2|
|  4|  2|   1|   1|
|  5|  1|   2|   2|
|  6|  2|   1|   2|
|  7|  2|   2|null|
+---+---+----+----+

+---------------+
|ConsecutiveNums|
+---------------+
|              1|
+---------------+

